# Geocoding in Python

Maps can be a powerful way to tell stories about your data, yet great datasets for geospatial analysis are rarely complete. At times you may have a dataset that only has an address column, missing key information you’ll need for spatial analysis – Latitude and Longitude coordinates. In this case you’ll need to do some __Geocoding__.

__Geocoding is the computational process of transforming a physical address description to a location on the Earth’s surface (spatial representation in numerical coordinates)__.

This sounds like a daunting process, but it can easily be done with the help of Python and the Bing Maps API. Before diving in, for any of this to work you will first have to acquire a Bing Maps API Key, which you can do by following these [simple steps](https://www.microsoft.com/en-us/maps/create-a-bing-maps-key "click this to get your API Key"). 

Next, import the libraries and packages needed which are the geocoder library and the pandas and re packages. The Geocoder package is the perfect candidate to solve this problem since it already deals with multiple geocoding services such as Google, Bing, OSM. Using this package and the Bing API Key, we’ll be able to use Bings data for our geocoding needs.

In [1]:
!pip install geocoder

In [2]:
import geocoder
import pandas as pd
import re
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

### Test The API

Now test out the Geocoder package and our API Key to make sure they work.

We’ll define our geocoder as g and make a request to Bing to geocode a specific address. The Bing API usually needs the address line(Street Address), locality(City/Town), and adminDistrict(State) to geocode the addresses. You also need to pass the developer key you retrieved from Bing through the geocoder.

After running this, your output should be the latitudinal and longitudinal coordinates of the address we used for testing.

In [3]:
APIKey= 'AhrrKF6BHmGIS3d71YtAb1AbGhXx5dBGhNgcWoS4vdd1apFoj1Spapvdq6r34UhC' 

In [4]:
g = geocoder.bing(None, addressline = '2831 Henderson Rd, Redding, CA 96002', locality='Redding', adminDistrict='California', method='details', key=APIKey)
g.json
for result in  g:
    print(result.latlng)

[40.57463837, -122.38108826]


As can be seen below, the same can be done to reverse geocode coordinates to retrieve specific address locations for any place in the world. 

In [5]:
g = geocoder.bing([[26.351670, 127.769400], [48.845580, 2.321807]], method='batch_reverse', key=APIKey)

for result in g:
    print(result.address, result.city, result.postal, result.state, result.country)

Kadena Town, Nakagami County, Okinawa, Japan Kadena Town  Okinawa Japan
114 Rue de Vaugirard, 75006 Paris, France Paris 75006 Île-de-France France


### Read In the Data

Now that its been confirmed that the geocoder tool is working, lets read in the excel file of the addresseses we'd like to geocode. 

In [6]:
df = pd.read_csv('addresses.csv')

In [7]:
df.head(5)

,Unnamed: 0,address
0,0,"Whitlow Plan, Brookland Grove, Washington, DC ..."
1,1,"Unit 02 Plan, Aria Reserve Miami, Miami, FL 33137"
2,2,"Unit 01 A Plan, Aria Reserve Miami, Miami, FL ..."
3,3,"The Buchanan Plan, The Townhomes At Michigan P..."
4,4,"Plan 3 Plan, Hudson At Belterra, Austin, TX 78737"


In [8]:
print(len(df))

196


In [9]:
add = df['address']
s= add.str.split(',')[1::2]
print(s[1][1])

 Aria Reserve Miami


Since the addresses are not formatted in the way necessary for the Bing API to do its work, some data transformation techniques will have to be employed.  Our data frame must have columns for the __address line__(Street Address), __locality__(City/Town), and  __adminDistrict__(State). 
We'll follow the steps below and use __Pandas__ as well as the __re__ packages to clean the data. 
1. Create and define a new Data frame outside of the forloop 
2. Use the re function to split our addresses by commas, which allow us to define the address, street, and city using their location each address string. 
3. Use the re function to split the addresses by spaces, which will allow us to extract the state and zip code using their location in the string
4. After defining all the and cleaning the key data needed, append the newly cleaned observations into the geo Data frame.

In [10]:
geo_data = []  # List to store the dictionaries

for i in df.address:
    s = i.split(',', 2)
    l = i.split()
    address = i
    street = s[0].strip()
    city = s[1].strip()
    country = 'U.S.'
    state = l[-2].strip()
    zipcode = l[-1].strip()
    
    # Append the data as a dictionary to the list
    geo_data.append({'address': address, 'street': street, 'city': city, 'state': state,
                     'zip': zipcode, 'country': country})

# Create a DataFrame from the list of dictionaries
geo = pd.DataFrame(geo_data)

In [17]:
geo.head()

,address,street,city,state,zip,country,lat,long
0,"Whitlow Plan, Brookland Grove, Washington, DC ...",Whitlow Plan,Brookland Grove,DC,20017,U.S.,38.904778,-77.016289
1,"Unit 02 Plan, Aria Reserve Miami, Miami, FL 33137",Unit 02 Plan,Aria Reserve Miami,FL,33137,U.S.,28.595512,-82.487343
2,"Unit 01 A Plan, Aria Reserve Miami, Miami, FL ...",Unit 01 A Plan,Aria Reserve Miami,FL,33137,U.S.,28.595512,-82.487343
3,"The Buchanan Plan, The Townhomes At Michigan P...",The Buchanan Plan,The Townhomes At Michigan Park,DC,20017,U.S.,38.904778,-77.016289
4,"Plan 3 Plan, Hudson At Belterra, Austin, TX 78737",Plan 3 Plan,Hudson At Belterra,TX,78737,U.S.,31.463848,-99.333298


Columns for the latitude and longitude coordinates the geocoder will retrieve from the the Bing API also needed to be created and added to the dataframe

In [18]:
geo['lat'] = ' '
geo['long'] = ' '

This is how the geo data frame should look like after our mini preprocessing session:

In [19]:
geo.head()

,address,street,city,state,zip,country,lat,long
0,"Whitlow Plan, Brookland Grove, Washington, DC ...",Whitlow Plan,Brookland Grove,DC,20017,U.S.,,
1,"Unit 02 Plan, Aria Reserve Miami, Miami, FL 33137",Unit 02 Plan,Aria Reserve Miami,FL,33137,U.S.,,
2,"Unit 01 A Plan, Aria Reserve Miami, Miami, FL ...",Unit 01 A Plan,Aria Reserve Miami,FL,33137,U.S.,,
3,"The Buchanan Plan, The Townhomes At Michigan P...",The Buchanan Plan,The Townhomes At Michigan Park,DC,20017,U.S.,,
4,"Plan 3 Plan, Hudson At Belterra, Austin, TX 78737",Plan 3 Plan,Hudson At Belterra,TX,78737,U.S.,,


### Geocoding The Addresses

Now its time to geocode. We will create a for loop to pass each address through the geocoder object created a couple steps above, except this time, we’ll be passing the column names for each iteration through it. Same as before, the Bing API Developer key will also have to be passed through the geocoder. We will then us the g.lat and g.long functions to request the latitudinal and longitudinal coordinates for all of our addresses.

In [14]:
for i in geo.index:
    try:
        g = geocoder.bing(None, addressline = geo['street'][i], locality= geo['city'][i], adminDistrict= geo['state'][i], PostalCode= geo['zip'][i], Countryregion= geo['country'][i], method='details', key='AqkhJ4a6kTQ6c4rm5vicHt7BsTav-KsWdaaKCnP794yFpzMDWPa1XCp14WUliC20')
        g.json
        geo.loc[i,'lat']=g.lat
        geo.loc[i, 'long']=g.lng
    except:
        geo.loc[i,'lat']=''
        geo.loc[i,'long']=''

Now lets take a look at the geo data frame

In [15]:
geo

,address,street,city,state,zip,country,lat,long
0,"Whitlow Plan, Brookland Grove, Washington, DC ...",Whitlow Plan,Brookland Grove,DC,20017,U.S.,38.904778,-77.016289
1,"Unit 02 Plan, Aria Reserve Miami, Miami, FL 33137",Unit 02 Plan,Aria Reserve Miami,FL,33137,U.S.,28.595512,-82.487343
2,"Unit 01 A Plan, Aria Reserve Miami, Miami, FL ...",Unit 01 A Plan,Aria Reserve Miami,FL,33137,U.S.,28.595512,-82.487343
3,"The Buchanan Plan, The Townhomes At Michigan P...",The Buchanan Plan,The Townhomes At Michigan Park,DC,20017,U.S.,38.904778,-77.016289
4,"Plan 3 Plan, Hudson At Belterra, Austin, TX 78737",Plan 3 Plan,Hudson At Belterra,TX,78737,U.S.,31.463848,-99.333298
...,...,...,...,...,...,...,...,...
191,"065-620-017, Redding, CA 96001",065-620-017,Redding,CA,96001,U.S.,40.574638,-122.381088
192,"(Undisclosed Address), Washington, DC 20020",(Undisclosed Address),Washington,DC,20020,U.S.,38.892063,-77.019913
193,"(Undisclosed Address), Miami, FL 33177",(Undisclosed Address),Miami,FL,33177,U.S.,25.775084,-80.194702
194,"(Undisclosed Address), Miami, FL 33125",(Undisclosed Address),Miami,FL,33125,U.S.,25.775084,-80.194702


We now have the coordinates for every single one of our addresses and have completed the geocoding process